In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_ctg.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-2-e74f7091776f>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
CTG,20210423,39.45,41.40,39.00,41.30,17558700,39.45,41.40,39.00,41.30,200000,1235300,540500,2021-04-23,16,2021,2021-16
CTG,20210422,41.75,41.95,39.15,39.15,21428500,41.75,41.95,39.15,39.15,0,222700,889500,2021-04-22,16,2021,2021-16
CTG,20210420,42.65,43.15,41.50,42.00,23241600,42.65,43.15,41.50,42.00,786000,79000,2853100,2021-04-20,16,2021,2021-16


In [3]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='CTG '))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
CTG,17.0,2015-05-04,2061730,2015-19,2015
CTG,18.0,2015-04-27,2147400,2015-18,2015
CTG,17.7,2015-04-24,1127530,2015-17,2015


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
CTG,39.15,2021-04-22,21428500,2021-16,2021,-5.205811
CTG,42.00,2021-04-20,23241600,2021-16,2021,7.279693
CTG,42.50,2021-04-19,16032000,2021-16,2021,1.190476


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    1499.000000
mean       -0.032328
std         2.201286
min        -6.525912
25%        -1.121082
50%         0.000000
75%         1.088141
max         7.522936
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    1499.000000
mean       22.431087
std         5.696222
min        14.600000
25%        18.500000
50%        20.750000
75%        24.300000
max        43.200000
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [25]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Rolling_volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(15).describe()

count    15.000000
mean     16.473813
std       3.983567
min      11.652100
25%      13.720750
50%      15.121500
75%      20.062150
max      23.241600
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean     16.531980
std       3.993474
min      11.652100
25%      14.031275
50%      14.886050
75%      20.079375
max      23.241600
Name: Volumn, dtype: float64

In [17]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
CTG,39.15,2021-04-22,21428500,2021-16,2021,-5.205811,21.4285
CTG,42.00,2021-04-20,23241600,2021-16,2021,7.279693,23.2416
CTG,42.50,2021-04-19,16032000,2021-16,2021,1.190476,16.0320
